<a href="https://colab.research.google.com/github/SoudeepGhoshal/ColabFiltering/blob/main/PIP_CollaborativeFiltering_MovieLens100k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import pairwise_distances
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cols_u = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users=pd.read_csv('/content/drive/MyDrive/ml-100k/u.user', sep='|', names=cols_u, encoding='latin-1', parse_dates=True)

ratings_u = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings=pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', sep='\t', names=ratings_u, encoding='latin-1')

movies_u = ['movie_id' , 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('/content/drive/MyDrive/ml-100k/u.item', sep='|', names=movies_u, usecols=range(5), encoding='latin-1')

movie_ratings=pd.merge(movies,ratings)
movie_df=pd.merge(movie_ratings,users)

movie_df.head(500)

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,timestamp,age,sex,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,60,M,retired,95076
...,...,...,...,...,...,...,...,...,...,...,...,...
495,214,Pink Floyd - The Wall (1982),01-Jan-1982,NaN,http://us.imdb.com/M/title-exact?Pink%20Floyd%...,148,5,877019882,33,M,engineer,97006
496,222,Star Trek: First Contact (1996),22-Nov-1996,NaN,http://us.imdb.com/M/title-exact?Star%20Trek:%...,148,4,877398901,33,M,engineer,97006
497,227,Star Trek VI: The Undiscovered Country (1991),01-Jan-1991,NaN,http://us.imdb.com/M/title-exact?Star%20Trek%2...,148,4,877399083,33,M,engineer,97006
498,228,Star Trek: The Wrath of Khan (1982),01-Jan-1982,NaN,http://us.imdb.com/M/title-exact?Star%20Trek:%...,148,4,877016514,33,M,engineer,97006


In [ ]:
#only for the normalization
min_rating = ratings['rating'].min()
max_rating = ratings['rating'].max()

ratings['rating'] = (ratings['rating'] - min_rating) / (max_rating - min_rating)

ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,0.50,881250949
1,186,302,0.50,891717742
2,22,377,0.00,878887116
3,244,51,0.25,880606923
4,166,346,0.00,886397596


In [ ]:
movie_df.drop(movie_df.columns[[3,4,7]], axis=1, inplace=True)
ratings.drop("timestamp", inplace=True, axis=1)
movies.drop(movies.columns[[3,4]], inplace=True, axis=1)
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   movie_id      100000 non-null  int64 
 1   title         100000 non-null  object
 2   release_date  99991 non-null   object
 3   user_id       100000 non-null  int64 
 4   rating        100000 non-null  int64 
 5   age           100000 non-null  int64 
 6   sex           100000 non-null  object
 7   occupation    100000 non-null  object
 8   zip_code      100000 non-null  object
dtypes: int64(4), object(5)
memory usage: 7.6+ MB


In [ ]:
user_input_percent = 0.3
subset_percent = 0.7

num_ratings = len(ratings)
num_user_input = int(num_ratings * user_input_percent)
num_subset = int(num_ratings * subset_percent)

user_input_ratings = ratings.sample(n=num_user_input, random_state=42)
subset_ratings = ratings.drop(user_input_ratings.index)
user_input_ratings

,user_id,movie_id,rating
75721,877,381,4
80184,815,602,3
19864,94,431,4
76699,416,875,2
92991,500,182,2
...,...,...,...
42648,385,919,4
86306,833,429,3
45466,246,550,3
63724,758,20,4


In [ ]:
# movies['title'] = movies['title'].str.replace(r'\(\d\d\d\d\)', '')
# movies['title'] = movies['title'].apply(lambda x: x.strip())
# movies.head()

In [ ]:
# userInput = [
#             {'title':'Breakfast Club, The', 'rating':5},
#             {'title':'Toy Story', 'rating':3.5},
#             {'title':'Jumanji', 'rating':2},
#             {'title':"Pulp Fiction", 'rating':5},
#             {'title':'Akira', 'rating':4.5}
#          ]
# inputMovies = pd.DataFrame(userInput)
# inputMovies

In [ ]:
# # inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]
# # inputMovies = pd.merge(inputId, inputMovies)
# inputMovies = inputMovies.drop('release_date', 1)
# inputMovies

In [ ]:
userSubset = subset_ratings[subset_ratings['movie_id'].isin(user_input_ratings['movie_id'].tolist())]
userSubset.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [ ]:
userSubsetGroup = userSubset.groupby(['user_id'])
userSubsetGroup.get_group(222)
mu_k=userSubsetGroup['rating'].mean()

In [ ]:
userSubsetGroup=sorted(userSubsetGroup,key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

<ipython-input-10-5489e10f2460>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup=sorted(userSubsetGroup,key=lambda x: len(x[1]), reverse=True)


[(405,
         user_id  movie_id  rating
  12383      405       592       1
  12449      405       171       1
  12460      405       580       1
  12607      405       387       1
  12731      405       190       2
  ...        ...       ...     ...
  98635      405      1475       1
  98717      405       513       1
  98956      405       375       1
  98978      405       445       4
  99756      405       958       1
  
  [458 rows x 3 columns]),
 (655,
         user_id  movie_id  rating
  39347      655       274       3
  39490      655       126       2
  39693      655       670       3
  39780      655       523       3
  39786      655       736       3
  ...        ...       ...     ...
  98813      655       155       4
  98839      655       558       4
  99071      655       143       4
  99365      655       649       3
  99418      655       305       4
  
  [443 rows x 3 columns]),
 (13,
         user_id  movie_id  rating
  63          13       526       3
  145     

In [ ]:
userSubsetGroup=userSubsetGroup[0:500]

In [ ]:
ratings_median=ratings['rating'].median()
print(ratings_median)

4.0


In [ ]:
def agreement(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    above_median1 = rating1 > rating_median
    below_median1 = rating1 < rating_median
    above_median2 = rating2 > rating_median
    below_median2 = rating2 < rating_median

    agreement_condition1 = np.logical_and(above_median1, below_median2)
    agreement_condition2 = np.logical_and(below_median1, above_median2)

    return np.logical_or(agreement_condition1, agreement_condition2).any()

def proximity(rating1, rating2, rating_max, rating_min, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    agree = agreement(rating1, rating2, rating_median)
    dist = np.where(agree, np.absolute(rating1 - rating2), 2 * np.absolute(rating1 - rating2))
    prox = ((2 * (rating_max - rating_min) + 1) - dist) ** 2
    return prox

def impact(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    impact_score = (np.absolute(rating1 - rating_median) + 1) * (np.absolute(rating2 - rating_median) + 1)
    agree = agreement(rating1, rating2, rating_median)
    impact_result = np.where(agree, impact_score, 1 / impact_score)
    return impact_result

def popularity(rating1, rating2, mu_k):
    pop = np.where(
        (rating1 > mu_k) & (rating2 > mu_k) | (rating1 < mu_k) & (rating2 < mu_k),
        1 + ((rating1 + rating2) / 2 - mu_k) ** 2,
        1
    )
    return pop

In [ ]:
import math
def agreement(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    above_median1 = rating1 > rating_median
    below_median1 = rating1 < rating_median
    above_median2 = rating2 > rating_median
    below_median2 = rating2 < rating_median

    agreement_condition1 = np.logical_and(above_median1, below_median2)
    agreement_condition2 = np.logical_and(below_median1, above_median2)

    return np.logical_or(agreement_condition1, agreement_condition2).any()

def proximity(rating1, rating2, rating_max, rating_min, rating_median):
    D = np.abs(rating1 - rating2)
    med_plus = (rating_max + rating_median) / 2
    med_minus = (rating_min + rating_median) / 2
    Rmax_minus_Rmin_squared = (rating_max - rating_min) ** 2

    agree = agreement(rating1, rating2, rating_median)
    prox = (D - med_plus + med_minus) ** 2 / Rmax_minus_Rmin_squared
    delta = 0.75 if np.abs(rating1 - rating2) > rating_median else (0.5 if np.abs(rating1 - rating2) == rating_median else 0.25)
    return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))

def impact(rating1, rating2, rating_median):
    agree = agreement(rating1, rating2, rating_median)
    impact1= math.exp(-1 / ((np.abs(rating1 - rating_median) + 1) * (np.abs(rating2 - rating_median) + 1)))
    impact2= math.exp(1 / ((np.abs(rating1 - rating_median) + 1) * (np.abs(rating2 - rating_median) + 1)))
    return impact1 if agree else impact2

def proposed_popularity(rating1, rating2,rIi):
    if (rating1 > rIi and rating2 > rIi) or (rating1 < rIi and rating2 < rIi):
        return math.log10(2 + (((rating1 + rating2) / 2) - rIi) ** 2)
    else:
        return 0.3010


In [ ]:
import numpy as np
import pandas as pd

def pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, rating_median, mu_k):
    pip_similarity_sum = 0
    n_ratings = 0

    for rating1, rating2 in zip(tempRatingList, tempGroupList):
        if not np.isnan(rating1) and not np.isnan(rating2):
            prox = proximity(rating1, rating2, rating_max, rating_min, rating_median)
            imp = impact(rating1, rating2, rating_median)
            pop = popularity(rating1, rating2,rating_median )

            pip_similarity_sum += prox * imp * pop
            n_ratings += 1

    if n_ratings == 0:
        return 0

    pip_similarity_value = pip_similarity_sum / n_ratings
    return pip_similarity_value

PIPdict = {}

rating_max = 5
rating_min = 0

for name, group in userSubsetGroup:

    group = group.sort_values(by='movie_id')
    inputMovies = user_input_ratings.sort_values(by='movie_id')
    temp_df = inputMovies[inputMovies['movie_id'].isin(group['movie_id'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    pip_similarity_value = pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, ratings_median, mu_k)
    PIPdict[name] = pip_similarity_value.mean()

print(PIPdict[1])

<ipython-input-14-d47755b63b66>:25: RuntimeWarning: divide by zero encountered in divide
  return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))


inf


In [ ]:
import numpy as np
import pandas as pd

def pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, rating_median, mu_k):
    pip_similarity_sum = 0
    n_ratings = 0

    for rating1, rating2 in zip(tempRatingList, tempGroupList):
        if not np.isnan(rating1) and not np.isnan(rating2):
            prox = proximity(rating1, rating2, rating_max, rating_min, rating_median)
            imp = impact(rating1, rating2, rating_median)
            pop = popularity(rating1, rating2,rating_median )

            pip_similarity_sum += prox * imp * pop
            n_ratings += 1

    if n_ratings == 0:
        return 0

    pip_similarity_value = pip_similarity_sum / n_ratings
    return pip_similarity_value

PIPdict = {}

rating_max = 5
rating_min = 0

for name, group in userSubsetGroup:
  group = group.sort_values(by='movie_id')
  inputMovies = user_input_ratings.sort_values(by='movie_id')
  temp_df = inputMovies[inputMovies['movie_id'].isin(group['movie_id'].tolist())]
  tempRatingList = temp_df['rating'].tolist()
  tempGroupList = group['rating'].tolist()

  similarity_values = []

  for temp_rating, group_rating in zip(tempRatingList, tempGroupList):
      pip_similarity_value = pip_similarity([temp_rating], [group_rating], rating_max, rating_min, ratings_median, mu_k)
      similarity_values.append(pip_similarity_value)

print(similarity_values)

<ipython-input-14-d47755b63b66>:25: RuntimeWarning: divide by zero encountered in divide
  return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))


[0.007788007830714049, 0.007788007830714049, 0.01648721270700128, 0.01648721270700128, 0.01648721270700128, inf, 0.007788007830714049, 0.01648721270700128, 0.01648721270700128, 0.01648721270700128, 0.01648721270700128, 0.01648721270700128, 0.01648721270700128, inf, 0.01648721270700128, 0.007788007830714049, 0.0042800847222924715, 0.01648721270700128, inf, 0.01648721270700128, 0.007788007830714049, inf, 0.07942472123261358, 0.0042800847222924715, inf, 0.01648721270700128, 0.01648721270700128, 0.01648721270700128, 0.01648721270700128, inf, inf, 0.007788007830714049, 0.007788007830714049, inf, inf, 0.01648721270700128, 0.01648721270700128, 0.03839421341813349, 0.01648721270700128, 0.01648721270700128, 0.008464817248906142]


In [ ]:
#PIP_DF = pd.DataFrame.from_dict(PIPdict, orient='index')
PIP_DF = pd.DataFrame(similarity_values, columns=['Similarity'])
PIP_DF.columns = ['similarityIndex']
PIP_DF['userId'] = PIP_DF.index
PIP_DF.index = range(len(PIP_DF))

#removing the inf values

PIP_DF['similarityIndex'] = PIP_DF['similarityIndex'].replace([np.inf, -np.inf], 0)
PIP_DF.head()

,similarityIndex,userId
0,0.007788,0
1,0.007788,1
2,0.016487,2
3,0.016487,3
4,0.016487,4


In [ ]:
topUsers=PIP_DF.sort_values(by='similarityIndex', ascending=False)
topUsers.head(100)

,similarityIndex,userId
22,0.079425,22
37,0.038394,37
12,0.016487,12
25,0.016487,25
27,0.016487,27
28,0.016487,28
19,0.016487,19
17,0.016487,17
35,0.016487,35
36,0.016487,36


In [ ]:
topUsersRating=topUsers.merge(ratings, left_on='userId', right_on='user_id', how='inner')
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
#topUsersRating.head(18000)
topUsersRating.head(1352)
topUsersRating.isnull().sum()

similarityIndex    0
userId             0
user_id            0
movie_id           0
rating             0
weightedRating     0
dtype: int64

In [ ]:
tempTopUsersRating = topUsersRating.groupby('movie_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
#handled the zero division error bug
recommendation_df.loc[tempTopUsersRating['sum_similarityIndex'] == 0, 'weighted average recommendation score'] = 0
recommendation_df.fillna(0, inplace=True)
##
recommendation_df['movieID'] = tempTopUsersRating.index
recommendation_df.tail()

,weighted average recommendation score,movieID
movie_id,,
1034,1.0,1034
1035,5.0,1035
1036,4.0,1036
1037,4.0,1037
1038,1.0,1038


In [ ]:
tempTopUsersRating.head(973)

,sum_similarityIndex,sum_weightedRating
movie_id,,
1,0.138635,0.556225
2,0.087213,0.182213
3,0.007788,0.031152
4,0.157442,0.738658
5,0.024275,0.072826
...,...,...
969,0.016487,0.049462
970,0.000000,0.000000
971,0.000000,0.000000


In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(100)

,weighted average recommendation score,movieID
movie_id,,
165,5.000000,165
119,5.000000,119
108,5.000000,108
1022,5.000000,1022
1019,5.000000,1019
...,...,...
523,4.725641,523
285,4.713412,285
99,4.698281,99


In [ ]:
recommendation_df = recommendation_df[(recommendation_df['weighted average recommendation score'] <= 5) &
 (recommendation_df['weighted average recommendation score'] >= 0) &
 (recommendation_df['weighted average recommendation score'].notnull())]
print(recommendation_df)

          weighted average recommendation score  movieID
movie_id                                                
593                                         5.0      593
614                                         5.0      614
601                                         5.0      601
768                                         5.0      768
617                                         5.0      617
...                                         ...      ...
908                                         0.0      908
909                                         0.0      909
910                                         0.0      910
911                                         0.0      911
364                                         0.0      364

[1029 rows x 2 columns]


In [ ]:
print(recommendation_df.isnull().sum())

weighted average recommendation score    0
movieID                                  0
dtype: int64


In [ ]:
recommendation_df.head(10)

,weighted average recommendation score,movieID
movie_id,,
593,5.0,593
614,5.0,614
601,5.0,601
768,5.0,768
617,5.0,617
753,5.0,753
224,5.0,224
362,5.0,362
581,5.0,581


In [ ]:
movies.loc[movies['movie_id'].isin(recommendation_df.head(10)['movieID'].tolist())]

,movie_id,title,release_date
223,224,Ridicule (1996),27-Nov-1996
361,362,Blues Brothers 2000 (1998),06-Feb-1998
580,581,Kalifornia (1993),01-Jan-1993
588,589,"Wild Bunch, The (1969)",01-Jan-1969
592,593,Stalingrad (1993),01-Jan-1993
600,601,For Whom the Bell Tolls (1943),01-Jan-1943
613,614,Giant (1956),01-Jan-1956
616,617,"Blue Angel, The (Blaue Engel, Der) (1930)",01-Jan-1930
752,753,Burnt By the Sun (1994),01-Jan-1994
767,768,Casper (1995),01-Jan-1995


In [ ]:
movies.head()

,movie_id,title,release_date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995


In [ ]:
from math import sqrt
merged_ratings = pd.merge(recommendation_df, ratings,on='movie_id')
merged_ratings['squared_diff'] = (merged_ratings['weighted average recommendation score'] - merged_ratings['rating']) ** 2
mean_squared_diff = merged_ratings['squared_diff'].mean()
mean_squared_diff
rmse_score = sqrt(mean_squared_diff)
print("RMSE Score:", rmse_score)

RMSE Score: 1.3545939448111404


In [ ]:
import pandas as pd
from math import sqrt
merged_ratings = pd.merge(recommendation_df, ratings, on='movie_id')

merged_ratings['abs_diff'] = abs(merged_ratings['weighted average recommendation score'] - merged_ratings['rating'])

mae_score = merged_ratings['abs_diff'].mean()

print("MAE Score:", mae_score)

MAE Score: 1.0160955739878432


In [ ]:
#addressing sparsity issue hereon

In [ ]:
userItemMatrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating')
userItemMatrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
null_values = userItemMatrix.isnull().sum().sum()
print(f"Total number of null values: {null_values}")


Total number of null values: 1486126


In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Create a copy of the userItemMatrix
userItemMatrix_imputed = userItemMatrix.copy()

# Calculate the total number of null values in the table
total_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the threshold for the number of null values to be imputed
null_threshold = int(total_null_values * 0.60)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')

# Iterate over each row
for index, row in userItemMatrix_imputed.iterrows():
    # Calculate the number of null values in each row
    row_null_values = row.isnull().sum()

    # If the number of null values in the row is less than the threshold, skip imputation for this row
    if row_null_values == 0 or row_null_values > null_threshold:
        continue

    # Create a mask for 60% of the null values in this row
    row_mask = np.random.choice([True, False], len(row), p=[0.6, 0.4])

    # Apply imputation for the selected 60% of null values in this row
    row_values = row.values
    row_values[row_mask] = imputer.fit_transform(row_values[row_mask].reshape(-1, 1)).ravel()

    # Update the values in the DataFrame for this row
    userItemMatrix_imputed.loc[index] = row_values

# Calculate the number of null values after imputation
remaining_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the number of null values that have been imputed
imputed_values = total_null_values - remaining_null_values

# Calculate the percentage of null values imputed
percent_imputed = (imputed_values / total_null_values) * 100

print(f"Percentage of null values imputed: {percent_imputed:.2f}%")
print(f"Remaining null values: {remaining_null_values}")

Percentage of null values imputed: 59.97%
Remaining null values: 594962


In [ ]:
#cpmare table sizes for any anomaly
print(userItemMatrix_imputed.shape[1])
print(userItemMatrix.shape[1])

1682
1682


In [ ]:
print(len(ratings))

100000


In [ ]:
ratings_series = userItemMatrix_imputed.stack()
ratings_imputed = ratings_series.reset_index()
ratings_imputed.columns = ['user_id', 'movie_id', 'rating']

In [ ]:
print(len(ratings_imputed))

991164


In [ ]:
#testing for newer values

In [ ]:
userItemMatrix_imputed

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,4.0,4.0,NaN,4.0,4.0,NaN,4.0,NaN,NaN,4.0
2,4.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,4.0,2.0,...,NaN,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,4.0
3,3.0,NaN,NaN,3.0,3.0,NaN,3.0,NaN,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,NaN,NaN,3.0,NaN,3.0
4,NaN,NaN,4.5,4.5,NaN,NaN,4.5,4.5,NaN,NaN,...,4.5,4.5,NaN,NaN,4.5,4.5,NaN,4.5,4.5,4.5
5,4.0,3.0,NaN,3.0,NaN,NaN,3.0,3.0,3.0,3.0,...,3.0,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.0,4.0,NaN,4.0,NaN,4.0,NaN,NaN,5.0,NaN,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
940,4.0,4.0,NaN,2.0,4.0,NaN,4.0,5.0,3.0,4.0,...,4.0,NaN,NaN,4.0,NaN,4.0,NaN,4.0,4.0,4.0
941,5.0,4.0,NaN,NaN,4.0,NaN,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,4.0,4.0,4.0,NaN,NaN,4.0


In [ ]:
user_input_percent = 0.3
subset_percent = 0.7

num_ratings = len(ratings_imputed)
num_user_input = int(num_ratings * user_input_percent)
num_subset = int(num_ratings * subset_percent)

user_input_ratings = ratings.sample(n=num_user_input, replace=True, random_state=42)
subset_ratings = ratings.drop(user_input_ratings.index)
user_input_ratings

,user_id,movie_id,rating
15795,222,200,0.50
860,284,322,0.50
76820,437,475,0.50
54886,343,276,1.00
6265,216,231,0.25
...,...,...,...
53466,545,29,0.50
154,299,111,0.50
74292,758,887,1.00
88583,18,28,0.50
